In [1]:
import casadi as ca
import numpy as np

In [2]:
def create_opt_problem(fi_func, N_xi, hi_func):
    xi = ca.SX.sym("xi",N_xi)
    fi = fi_func(xi)
    g = hi_func(xi)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':xi, 'g':g, 'f':fi}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

In [3]:
def create_subproblem(fi_func, Ai, rho, hi_func):
    N_lambda, N_yi = np.shape(Ai)
    yi = ca.SX.sym("yi",N_yi)
    xi = ca.SX.sym("xi",N_yi)
    sigma_i = ca.SX.sym('sigma_i',N_yi,N_yi)
    lambda_ = ca.SX.sym("lambda",N_lambda)
    
    fi = fi_func(yi) + lambda_.T @ Ai @ yi + rho/2 * (yi - xi).T @ sigma_i @ (yi - xi)
    p = ca.vertcat(lambda_, xi, ca.reshape(sigma_i, -1,1))
    g = hi_func(yi)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':yi, 'g':g, 'f':fi, 'p': p}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

In [4]:
def constraint_jac_approx(yi, hi_func, hi_jac_func):
    constraint_res = hi_func(yi)    #  Residue
    Nh = np.shape(constraint_res)[0]
    Ny = np.shape(yi)[0]
    zero_row = ca.DM.zeros(1,Ny)
    hi_jac = hi_jac_func(yi)
    for i in range(Nh):
        if constraint_res[i] != 0:    #  TODO: deal with small value
            hi_jac[i,:] = zero_row
    hi_jac = ca.DM.zeros(Nh,Ny)
    return hi_jac

In [5]:
def modified_grad(fi_grad, hi_jac_approx, hi_jac_real, kappa_i):
    return fi_grad + (hi_jac_real - hi_jac_approx).T @ kappa_i

In [6]:
# # Test QP subprobelm
# A_list = []
# A = ca.diag([1,1])
# A = ca.DM([[1,1],[1,1]])
# A_list += [A]
# b = ca.DM([0,0])
# mu = 1e5
# N_hi_list = [1]
# QP_solver = create_QP_problem(A_list, b,  mu, N_hi_list)
# delta_yi_list = [0.1,-0.2]
# s_list = [0,0]

# nl_qp = {}
# nl_qp['lbg'] = [0] * (3)
# nl_qp['ubg'] = [0] * (3)
# nl_qp['lbx'] =  [-np.inf] * (2+2)    # delta_y and s lower bound
# nl_qp['ubx'] =  [+np.inf] * (2+2)    # delta_y and s upper bound

# nl_qp['x0'] = delta_yi_list + s_list    #  Initial guess

# H_para = ca.reshape(ca.diag([2,2]),-1,1)
# modified_grad_value = ca.DM([1,1])
# y = ca.DM([-2,-2])
# Ci = ca.DM.zeros(2,1)
# Ci = ca.DM([3,1])
# lambda_ = ca.DM.zeros(2,1)
# p = ca.vertcat(lambda_, H_para, modified_grad_value, y, Ci)
# nl_qp['p'] = ca.DM(p)
# QP_sol = QP_solver(**nl_qp)

# QP_sol

In [7]:
def create_QP_problem(A_list, b,  mu, N_hi_list):
    N = len(A_list)
    N_lambda = np.shape(A_list[0])[0]
    
    s = ca.SX.sym("s", N_lambda)
    lambda_ = ca.SX.sym("lambda_", N_lambda)
    
    delta_yi_list = []
    fkh_hess_col_list = [] 
    modiefied_grad_col_list = []
    Ci_col_list = []
    
    yi_list = []
    obj = 0
    sigma_Ai = 0
    g = []
    for i in range(N):
        Ai = A_list[i]
        N_delta_yi = np.shape(Ai)[1]
        Hi = ca.SX.sym("Hi" + str(i), N_delta_yi, N_delta_yi)
        gi = ca.SX.sym("gi" + str(i), N_delta_yi)
        yi = ca.SX.sym("yi" + str(i), N_delta_yi)
        Ci = ca.SX.sym("Ci" + str(i), N_hi_list[i], N_delta_yi)
        
        fkh_hess_col_list += [ca.reshape(Hi, -1, 1)]
        modiefied_grad_col_list += [ca.reshape(gi, -1, 1)]
        yi_list += [yi]
        
        delta_yi = ca.SX.sym("delta_yi" + str(i),N_delta_yi)
        delta_yi_list += [delta_yi]
    
        obj += 1/2 * delta_yi.T @ Hi @ delta_yi + gi.T @ delta_yi
        sigma_Ai += Ai @ (yi + delta_yi)
        
        Ci_col_list += [ca.reshape(Ci, -1, 1)]
        g += [Ci @ delta_yi]
    obj += lambda_.T @ s + mu/2 * s.T @ s
    x = ca.vertcat(*delta_yi_list, s)
    p = ca.vertcat(lambda_, *(fkh_hess_col_list + modiefied_grad_col_list + yi_list + Ci_col_list))

    g += [ sigma_Ai - b - s ]
    g = ca.vertcat(*g)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':x, 'g':g, 'f':obj, 'p': p}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver    

### Remark
1. The quality of solution depends to a large degree on the $\sigma$
2. As N increase, the solution of y divergents under large $\mu$ (1e5).
3. minimal $\log_{10} (\|x -x ^{*}\|_{\infty})$ is -0.0803939. Cannot be arbitrarily small.

In [8]:
np.random.seed(1)
N = 2500 # Should test with 25000
Nx = 4
eps = 1e-5
# sigma_i = 1
# sigma_bar_i = 1
sigma_i = 0.5
sigma_bar_i = 0.5
# rho = 0.75
rho = 1
N_itermax = 10

# Define A matrix
A_list = []
NA_col = int(N * Nx / 2)
I = ca.diag([1,1])
for i in range(N):
    A = ca.DM.zeros(NA_col, Nx)
    if i == 0:
        A[NA_col-2: , :2] = -I
    else:
        A[(i-1)*2: (i-1)*2+2, :2] = -I        
    A[i*2:i*2+2,Nx-2:] = I
    A_list += [A]
# Define b
b = ca.DM.zeros(NA_col,1)

In [9]:
# Define parameter
eta_list = []
eta_nom_list = []
eta_bar_list = []
eta_bar_nom_list = []

for i in range(N):
    eta_nom = np.array([[N * np.cos(2*(i+1) * np.pi / N)],[N * np.sin(2*(i+1) * np.pi / N)]])
    eta_nom_list += [eta_nom]
    eta = eta_nom + sigma_i * np.random.randn(2,1)
    eta_list +=  [ca.DM(eta)]
    
    eta_bar_nom = 2 * N * np.sin(np.pi / N)
    eta_bar_nom_list +[eta_bar_nom]
    eta_bar =  eta_bar_nom + sigma_i * np.random.randn(1)
    eta_bar_list += [ca.DM(eta_bar)]


In [10]:
# Define obejective function
fi_list = []
fi_func_list = []
x = ca.SX.sym("x",Nx)    #  $x_{i}=\left(\chi_{i}^{\top}, \zeta_{i}^{\top}\right)^{\top} \in \mathbb{R}^{4}$
for i in range(N):
    if i == N-1:
        fi = 1 / (4 * sigma_i ** 2) * (x[0:2] - eta_list[i]).T @ (x[0:2] - eta_list[i]) + 1 / (4 * sigma_i**2) * (x[2:] - eta_list[0]).T @ (x[2:] - eta_list[0]) + 1 / (2 * sigma_bar_i**2) * ( ca.norm_2(x[0:2] - x[2:]) - eta_bar_list[i] )**2
#         fi = 1 / (4 * sigma_i ** 2) * (x[0:2] - eta_list[i]).T @ (x[0:2] - eta_list[i]) + 1 / (4 * sigma_i**2) * (x[2:] - eta_list[0]).T @ (x[2:] - eta_list[0]) 
    else:
        fi = 1 / (4 * sigma_i ** 2) * (x[0:2] - eta_list[i]).T @ (x[0:2] - eta_list[i]) + 1 / (4 * sigma_i**2) * (x[2:] - eta_list[i+1]).T @ (x[2:] - eta_list[i+1]) + 1 / (2 * sigma_bar_i**2) * (ca.norm_2(x[0:2] - x[2:]) - eta_bar_list[i])**2
#         fi = 1 / (4 * sigma_i ** 2) * (x[0:2] - eta_list[i]).T @ (x[0:2] - eta_list[i]) + 1 / (4 * sigma_i**2) * (x[2:] - eta_list[i+1]).T @ (x[2:] - eta_list[i+1]) 
    fi_list += [fi]
    fi_func = ca.Function("fi_func", [x], [fi])
    fi_func_list += [fi_func]

In [11]:
# fi_list

In [12]:
# Define gradtient function
fi_grad_list = []
fi_grad_func_list = []
for i in range(N):
    fi_grad = ca.gradient(fi_list[i], x)
    fi_grad_list += [fi_grad]
    fi_grad_func = ca.Function("fi_grad_func", [x], [fi_grad])
    fi_grad_func_list += [fi_grad_func]

In [13]:
# Define inequality constraints
hi_list = []
hi_func_list = []
Nhi_list = []
for i in range(N):
    hi = (ca.norm_2(x[0:2] - x[2:]) - eta_bar_list[i])**2 - sigma_bar_i ** 2
#     hi = ca.DM(0)
    hi_list += [hi]
    hi_func = ca.Function("hi_func", [x], [hi])
    hi_func_list += [hi_func]
    # Deal with the number of inequality constraints for each i.
    Nhi = np.shape(hi)[0]
    Nhi_list += [Nhi]

In [14]:
# Define approximate jacobian, real jacobian and Hessian.
kappa_i_list = []
hi_jac_list = []
hi_jac_func_list = []
fkh_hess_i_list = []
fkh_hess_func_list = []
for i in range(N):
    # Kappa
    kappa_i = ca.SX.sym("kappa_i",Nhi)
    kappa_i_list += [kappa_i]
    # Jacobian function
    hi_jac = ca.jacobian(hi_list[i],x)
    hi_jac_list += [hi_jac]
    hi_jac_func = ca.Function("hi_jac_func",[x],[hi_jac])
    hi_jac_func_list +=  [hi_jac_func]
    # Hessian fucntion
    fi = fi_list[i]
    hi = hi_list[i]
    fkh_i = fi + kappa_i.T @ hi
    fkh_hess_i = ca.hessian(fkh_i, x)[0]
    fkh_hess_i_func = ca.Function("fkh_hess_i_func", [x, kappa_i], [fkh_hess_i])
    fkh_hess_i_list += [fkh_hess_i]
    fkh_hess_func_list += [fkh_hess_i_func]

In [15]:
subsolver_list = []
# Define subproblem solvers
for i in range(N):
    Ai = A_list[i]
    fi_func = fi_func_list[i]
    hi_func = hi_func_list[i]
    subsolver_list += [create_subproblem(fi_func, Ai, rho, hi_func)]

In [16]:
opt_solver_list = []
for i in range(N):
    Ai = A_list[i]
    N_xi = np.shape(Ai)[1]
    fi_func = fi_func_list[i]
    hi_func = hi_func_list[i]
    opt_solver_list += [create_opt_problem(fi_func, N_xi, hi_func)]

In [17]:
# Define QP problem
mu = 10
QP_solver = create_QP_problem(A_list, b,  mu, Nhi_list)

In [18]:
# Initial guess
delta_yi_list = []
Sigma_i_list = []
xi_list = []
yi_list = []
lbhi_list = []
ubhi_list = []
lbx_list = []
ubx_list = []
Nx = 0
Nhi_sum = 0
for i in range(N):
    Ai = A_list[i]
    N_lambda, N_xi = np.shape(Ai)
    Nx += N_xi
    Nhi = Nhi_list[i]
    Nhi_sum += Nhi
    xi = np.random.randn(N_xi,1).flatten().tolist()
# #     xi = ca.DM.zeros(N_xi,1).full().flatten().tolist()
#     if i == N-1:
#         xi = ca.vertcat(eta_list[N-1],eta_list[0]).full().flatten().tolist()
#     else:
#         xi = ca.vertcat(eta_list[i],eta_list[i+1]).full().flatten().tolist()
    xi_list += [xi]

#     print(N_xi)
    Sigma_i_list += [ca.diag([1] * N_xi)]
    
    lbhi_list += [[-ca.inf] * Nhi]
    ubhi_list += [[0] * Nhi]
    yi = np.random.randn(N_xi,1).flatten().tolist()
#     yi = xi
    yi_list += [yi]   
#     yi_list += [[0] * N_xi]
    lbx_list += [[-ca.inf] * N_xi]
    ubx_list += [[ca.inf] * N_xi]
lambda_ = np.random.randn(N_lambda,1)
lambda_ = ca.DM(lambda_)
# lambda_ = ca.DM.zeros(N_lambda,1)
s_list = [0] * N_lambda
delta_yi_list = sum(yi_list,[])

In [19]:
# yi_list

In [20]:
# Define opt solver
xi_opt_sol_list = []

nl_opt = {}
nl_opt['x0'] = []
for i in range(N):
    Ai = A_list[i]
    N_lambda_i, N_xi = np.shape(Ai)
    nl_opt['lbg'] = lbhi_list[i]
    nl_opt['ubg'] = ubhi_list[i]    
    
    if i == N-1:
        x0 = ca.vertcat(eta_list[N-1],eta_list[0]).full().flatten().tolist()
    else:
        x0 = ca.vertcat(eta_list[i],eta_list[i+1]).full().flatten().tolist()
    
    nl_opt['x0'] = x0
    nl_opt['lbx'] = lbx_list[i]
    nl_opt['ubx'] = ubx_list[i]
    solver_optproblem = opt_solver_list[i]
    xi_opt_sol = solver_optproblem(**nl_opt)
#         print(yi_sol)
    xi_opt_sol_list += [xi_opt_sol['x'].full().flatten().tolist().copy()]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [21]:
# xi_opt_sol_list

In [22]:
# Define solver
nl_sub = {}

nl_qp = {}
nl_qp['lbg'] = [0] * (Nhi_sum + N_lambda)
nl_qp['ubg'] = [0] * (Nhi_sum + N_lambda)
nl_qp['lbx'] = sum(lbx_list,[]) + [-np.inf] * N_lambda    # delta_y and s lower bound
nl_qp['ubx'] = sum(ubx_list,[]) + [+np.inf] * N_lambda    # delta_y and s upper bound

In [23]:
# Track solution
yi_sol_list = []
delta_y_sol_list = []
lambda_list = []
x_sol_list = []
s_sol_list = []
x_sol_list += [xi_list.copy()]
lambda_list += lambda_.full().flatten().tolist()

In [24]:
# lambda_

In [25]:
# solve problem

for i in range(N_itermax):
    sum_Ay = 0
    kappa_sol_list = []
    # Step 1: solve the subproblem      
    for j in range(N):
        Ai = A_list[j]
        N_lambda_i, N_xi = np.shape(Ai)
        Sigma_i = Sigma_i_list[j]
        nl_sub['lbg'] = lbhi_list[j]
        nl_sub['ubg'] = ubhi_list[j]    
        nl_sub['x0'] = yi_list[j]
        nl_sub['lbx'] = lbx_list[j]
        nl_sub['ubx'] = ubx_list[j]
        nl_sub['p'] = lambda_list + xi_list[j] + ca.reshape(Sigma_i, -1, 1).full().flatten().tolist()
#         print(nl_sub)
        solver_subproblem = subsolver_list[j]
        yi_sol = solver_subproblem(**nl_sub)
#         print(yi_sol)
        yi_list[j] = yi_sol['x'].full().flatten().tolist()
#         print("yi_sol",yi_sol['x'])
        yi_sol_list += [yi_list[j].copy()]
        kappa_i_sol = yi_sol['lam_g']
        kappa_sol_list += [kappa_i_sol]
        
        sum_Ay += Ai @ yi_sol['x']

    # Step 2: Check if the tolerance satisfied
    #TODO: modify
#     N_flag = 0
#     for j in range(N):
#         if rho * ca.norm_1( Sigma_i_list[j] @ ca.DM(yi_list[j])) <= eps:
#             N_flag += 1
#     if ca.norm_1(sum_Ay - b) <= eps and N_flag == N:
#         break
    # Step3: update Jacobian approximations, calculate the modified gradient, and update Hessian
    Ci_list = []    #  constraint Jacobian
    g_list = []    #  modified gradient
    H_list = []    #  Hessian
    for j in range(N):
        # 3.1 Choose Jacobian approximations
        yi = yi_list[j]
        hi_func = hi_func_list[j]
        hi_jac_func = hi_jac_func_list[j]
        fkh_hess_func = fkh_hess_func_list[j]
        hi = hi_func(yi)
        kappa_i_sol = kappa_sol_list[j]
        fi_grad = fi_grad_func_list[j](yi)
        hi_jac_real = hi_jac_func(yi)
        
        hi_jac_approx = constraint_jac_approx(yi, hi_func, hi_jac_func)
        Ci_list += [ca.reshape(hi_jac_approx, -1, 1)]
        gi = modified_grad(fi_grad, hi_jac_approx, hi_jac_real, kappa_i_sol)
        g_list += [ca.reshape(gi, -1, 1)]
        
        Hi = fkh_hess_func(yi, kappa_i_sol)
        H_list += [ca.reshape(Hi, -1, 1)]
#         print("hi", hi, "kappa_i_sol",kappa_i_sol, "fi_grad",fi_grad, "hi_jac_real",hi_jac_real, "hi_jac_approx",hi_jac_approx, "gi",gi, "Hi" ,Hi)
    # Step 4: Solve QP problem
    nl_qp['x0'] = delta_yi_list + s_list    #  Initial guess
    
    H_para = ca.vertcat(*H_list)
    modified_grad_value = ca.vertcat(*g_list)
    y = ca.vertcat(* sum(yi_list,[]))
    Ci = ca.vertcat(*Ci_list)
    lambda_ = ca.vertcat(lambda_list)
    p = ca.vertcat(lambda_, H_para, modified_grad_value, y, Ci)
    nl_qp['p'] = ca.DM(p)
#     print("lambda_",lambda_)
#     print("H_para", H_para)
#     print("modified_grad_value", modified_grad_value)
#     print("y",y)
#     print("Ci",Ci)
    QP_sol = QP_solver(**nl_qp)
#     print(QP_sol)
    alpha1 = 1
    alpha2 = 1
    alpha3 = 1
    # Step 5: Update x and lambda
    pos = 0
#     print(QP_sol['x'][0:Nx,:])
    delta_y = QP_sol['x'][0:Nx,:]
    delta_y_sol_list += [delta_y]
    s_sol = QP_sol['x'][Nx:,:]
    s_sol_list += [s_sol]
    QP_list = QP_sol['x'].full().flatten().tolist()
    lambda_QP = QP_sol['lam_g'][:N_lambda]
#     print("lambda_QP", lambda_QP)
    x = ca.DM(sum(xi_list,[]))
    y = ca.DM(sum(yi_list,[]))
#     print(delta_y)
#     print(y,x)
    x_plus = x + alpha1 * (y - x) + alpha2 * delta_y
    for j in range(N):
        list_len = len(xi_list[j])
        xi_list[j] = x_plus[pos:pos+list_len].full().flatten().tolist()
        pos = pos+list_len
#     print(xi_list)
#     print(lambda_QP)
    lambda_ = lambda_ + alpha3 * (lambda_QP - lambda_)
    lambda_list = lambda_.full().flatten().tolist()
    x_sol_list += [xi_list.copy()]

In [26]:
# xi_opt_sol_list

In [27]:
err = []
for i in range(N_itermax+1):
    x_sol_opt_col = ca.reshape(ca.DM(xi_opt_sol_list),-1,1)
    x_temp_col = ca.reshape(ca.DM(x_sol_list[i]),-1,1)
    err += [ca.log10(ca.norm_inf(x_sol_opt_col-x_temp_col))]
err

[DM(3.3987),
 DM(0.226056),
 DM(-0.0588256),
 DM(-0.0841398),
 DM(-0.080111),
 DM(-0.0804331),
 DM(-0.0803909),
 DM(-0.0803942),
 DM(-0.0803938),
 DM(-0.0803939),
 DM(-0.0803939)]

In [28]:
# eta_list

In [29]:
# eta_nom_list

In [30]:
# lambda_list

In [31]:
# yi_sol_list

In [32]:
# s_sol_list

In [33]:
# delta_y_sol_list